<a href="https://colab.research.google.com/github/MykolaPrykhodko/MykolaPrykhodko/blob/main/mitoTest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [50]:
import pandas as pd
import numpy as np

## Підключення хмарного середовища зберігання довідників для розрахунку

In [51]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


## Для читання потрібна певна авторізація

###Імпортування бібліотек для аутентифікації та встановлення зв'язку із Таблицями.

In [52]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

In [53]:
path_in ='/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/Out/fop.xlsx'

In [54]:
sheet_df_dictonary = pd.read_excel(path_in, engine='openpyxl', sheet_name=['Sheet1'], skiprows=0)

In [55]:
Sheet1 = sheet_df_dictonary['Sheet1'].drop(['Unnamed: 0'], axis=1)

In [56]:
print(Sheet1.columns)

Index(['Таб №', 'Місяць бюджетного року', 'ПІБ', 'ЦФУ код', 'ЦФУ назва',
       '5.4.1.1.0.', '5.4.1.1.8.', '5.4.1.1.1.', '5.4.1.1.2.', '5.4.1.1.3.3.',
       '5.4.1.1.3.5.', '5.4.1.1.5.', '5.4.1.1.4.', '5.4.1.3.', '5.4.1.1.7.',
       '5.4.1.2.1.', '5.4.1.5.8.'],
      dtype='object')


In [57]:
df_new = Sheet1[['Місяць бюджетного року', 'ЦФУ код', 'ЦФУ назва', '5.4.1.1.0.']].copy()

## Початковий масив

In [58]:
df_tmp = df_new[df_new['Місяць бюджетного року'] == 'січень']
result = df_tmp.groupby(['ЦФУ код', 'ЦФУ назва'])['5.4.1.1.0.'].aggregate('sum').to_frame().reset_index()

In [59]:
result.insert(0, 'Назва статті', 'основна заробітна плата (оклади) - бюджет')
result.insert(0, 'Код статті', '5.4.1.1.0.')

In [60]:
result_total = result.rename(columns = {'5.4.1.1.0.':'січень'})

In [61]:
result_total.head()

,Код статті,Назва статті,ЦФУ код,ЦФУ назва,січень
0,5.4.1.1.0.,основна заробітна плата (оклади) - бюджет,0.17..,(до 01.01.2018) Управління депозитарної дiяльн...,114593
1,5.4.1.1.0.,основна заробітна плата (оклади) - бюджет,0.23.1.1.,Сектор розрахунків з персоналом,55057
2,5.4.1.1.0.,основна заробітна плата (оклади) - бюджет,0.27.,Перший відділ,60752
3,5.4.1.1.0.,основна заробітна плата (оклади) - бюджет,1.05.11.2.,Відділ аналізу кредитного ризику та звітності,132414
4,5.4.1.1.0.,основна заробітна плата (оклади) - бюджет,1.05.13.,Управління оцінки та моніторингу забезпечення,62409


In [62]:
months = ['лютий', 'березень', 'квітень', 'травень', 'червень', 'липень', 'серпень', 'вересень', 'жовтень', 'листопад', 'грудень']

In [63]:
for i in months:
  df_new_tmp = df_new[df_new['Місяць бюджетного року'] == i]
  result = df_new_tmp.groupby(['ЦФУ код', 'ЦФУ назва'])['5.4.1.1.0.'].aggregate('sum').to_frame().reset_index()
  result.insert(0, 'Назва статті', 'основна заробітна плата (оклади) - бюджет')
  result.insert(0, 'Код статті', '5.4.1.1.0.')
  # result_total = result.rename(columns = {'5.4.1.1.0.': i})
  df_tmp = result.rename(columns = {'5.4.1.1.0.': i})
  result_total = pd.merge(result_total, df_tmp)

In [64]:
result_total.head()

,Код статті,Назва статті,ЦФУ код,ЦФУ назва,січень,лютий,березень,квітень,травень,червень,липень,серпень,вересень,жовтень,листопад,грудень
0,5.4.1.1.0.,основна заробітна плата (оклади) - бюджет,0.17..,(до 01.01.2018) Управління депозитарної дiяльн...,114593,114593,114593,114593,114593,114593,114593,114593,114593,114593,114593,114593
1,5.4.1.1.0.,основна заробітна плата (оклади) - бюджет,0.23.1.1.,Сектор розрахунків з персоналом,55057,55057,55057,55057,55057,55057,55057,55057,55057,55057,55057,55057
2,5.4.1.1.0.,основна заробітна плата (оклади) - бюджет,0.27.,Перший відділ,60752,60752,60752,60752,60752,60752,60752,60752,60752,60752,60752,60752
3,5.4.1.1.0.,основна заробітна плата (оклади) - бюджет,1.05.11.2.,Відділ аналізу кредитного ризику та звітності,132414,132414,132414,132414,132414,132414,132414,132414,132414,132414,132414,132414
4,5.4.1.1.0.,основна заробітна плата (оклади) - бюджет,1.05.13.,Управління оцінки та моніторингу забезпечення,62409,62409,62409,62409,62409,133691,133691,133691,133691,133691,133691,133691


### Для січень

### Лютий

In [22]:
# path_out = '/content/gdrive/Shareddrives/Фінансовий департамент/Управління фінансового планування/Бюджет 2023/Budget FOP/Out/fop_SA.xlsx'

In [23]:
# table.to_excel(path_out)

In [24]:
# # Changed Таб № to dtype str
# Sheet1['Таб №'] = Sheet1['Таб №'].astype('str')

# # Changed 5.1.1.0. to dtype float
# Sheet1['5.1.1.0.'] = Sheet1['5.1.1.0.'].astype('float')

# # Changed 5.4.1.1.2. to dtype float
# Sheet1['5.4.1.1.2.'] = Sheet1['5.4.1.1.2.'].astype('float')

# # Changed 5.4.1.1.3.3. to dtype float
# Sheet1['5.4.1.1.3.3.'] = Sheet1['5.4.1.1.3.3.'].astype('float')

# # Changed 5.4.1.1.3.5. to dtype float
# Sheet1['5.4.1.1.3.5.'] = Sheet1['5.4.1.1.3.5.'].astype('float')

# # Changed 5.4.1.1.5. to dtype float
# Sheet1['5.4.1.1.5.'] = Sheet1['5.4.1.1.5.'].astype('float')

# # Changed 5.4.1.1.4. to dtype float
# Sheet1['5.4.1.1.4.'] = Sheet1['5.4.1.1.4.'].astype('float')

# # Changed 5.4.1.3. to dtype float
# Sheet1['5.4.1.3.'] = Sheet1['5.4.1.3.'].astype('float')

In [25]:
print(Sheet1.head(2))

  Таб № Місяць бюджетного року                         ПІБ ЦФУ код  \
0  1811                 січень  Кравчук Лариса Анатоліївна   1.38.   
1  1811                  лютий  Кравчук Лариса Анатоліївна   1.38.   

                                           ЦФУ назва  5.4.1.1.0.  5.4.1.1.8.  \
0  Департамент корпоративного банкінгу та транзак...       38800      388000   
1  Департамент корпоративного банкінгу та транзак...       38800      543200   

   5.4.1.1.1.  5.4.1.1.2.  5.4.1.1.3.3.  5.4.1.1.3.5.  5.4.1.1.5.  5.4.1.1.4.  \
0     -349200      3600.0             0          2000           0           0   
1     -504400      3600.0         25000          2000           0       25777   

   5.4.1.3.  5.4.1.1.7.  5.4.1.2.1.    5.4.1.5.8.  
0    142827  1370.50164     25410.0  10700.375082  
1    494046  4242.40560     25410.0  29654.920280  
